In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import numpy as np
import tensorflow as tf
import os
#os.chdir('/m/nbe/project/rtmeg/problearn/mneflow/')
os.chdir('/u/20/vranoug1/unix/OPM-BCI/mneflow-dev/mneflow/')
import mneflow
from sklearn.preprocessing import MinMaxScaler

path = '/m/nbe/work/vranoug1/OPM-BCI/datasets/'
fname = 'sim_2_erps_50nA.npz'


import_opt = dict(fs=500,
                  savepath='../tfr/',
                  out_name=fname+'0',
                  input_type='trials',
                  target_type='float',
                  val_size=0.2,
                  array_keys={'X': 'X', 'y': 'y'},
                  # picks=None, target_picks=None,
                  scale=True,
                  scale_interval=None,
                  # crop_baseline=True,
                  decimate=None,
                  bp_filter=False,
                  overwrite=False,
                  test_set='holdout')

meta = mneflow.produce_tfrecords([path+fname], **import_opt)
dataset = mneflow.Dataset(meta, train_batch=100, class_subset=None, pick_channels=None, decim=None)

W0212 17:55:09.811821 140673010104064 deprecation.py:323] From /u/20/vranoug1/unix/.conda/envs/py3ml2/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


Metadata file found, restoring


W0212 17:55:13.254986 140673010104064 deprecation.py:323] From /u/20/vranoug1/unix/.conda/envs/py3ml2/lib/python3.7/site-packages/mneflow/data.py:122: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [2]:
#optimizer_params = dict(l1_lambda=3e-4, learn_rate=3e-4, task='regression')
optimizer_params = dict(l2_lambda=3e-2, l1_scope=['fc'], l1_lambda=3e-4,
                        l2_scope=['dmx', 'tconv'], learn_rate=3e-4,
                        task='regression') # l1_scope=['fc'], , l2_scope=['dmx', 'tconv']

optimizer = mneflow.Optimizer(**optimizer_params)

# specify model parameters
lf_params = dict(n_ls=32,  # number of latent factors
                 filter_length=64,  # convolutional filter length
                 pooling=32,  # convlayer pooling factor
                 stride=16,  # stride parameter for pooling layer
                 padding='SAME',
                 dropout=.5,
                 nonlin=tf.nn.relu,
                 pool_type='max',
                 model_path = import_opt['savepath'],  # not used at the moment
                )

# 1. LFCNN

In [ ]:
model = mneflow.models.LFCNN(dataset, optimizer, lf_params)
model.build()

model.train(n_iter=30000, eval_step=500, min_delta=1e-6, early_stopping=6)

X0: (?, 102, 300)
de-mix init : OK
dmx (?, 102, 1, 32)
conv init : OK
f: (64, 1, 32, 1)
lf-out (?, 7, 1, 32)
fc ::: 224 1
fc init : OK
X: (?, 102, 300, 1)
y_pred: (?, 1)
(1,)
L1 penalty applied to fc
L2 penalty applied to dmx, tconv
Initialization complete!
i 0, tr_loss 2.12987, tr_acc -14.7485 v_loss 2.05016, v_acc -15.3524
i 500, tr_loss 0.804355, tr_acc -0.00663733 v_loss 0.822143, v_acc -0.205144
i 1000, tr_loss 0.548503, tr_acc 0.107227 v_loss 0.57138, v_acc -0.0625696
i 1500, tr_loss 0.356369, tr_acc 0.30823 v_loss 0.391204, v_acc -0.0885206
i 2000, tr_loss 0.235809, tr_acc 0.320915 v_loss 0.263893, v_acc -0.0624317
i 2500, tr_loss 0.157156, tr_acc 0.249068 v_loss 0.184154, v_acc -0.0654293
i 3000, tr_loss 0.119255, tr_acc 0.324151 v_loss 0.134703, v_acc -0.0291443
i 3500, tr_loss 0.0733857, tr_acc 0.44361 v_loss 0.110134, v_acc -0.0551417
i 4000, tr_loss 0.0716364, tr_acc 0.393894 v_loss 0.109452, v_acc -0.256132
i 4500, tr_loss 0.0468836, tr_acc 0.559103 v_loss 0.0918804, v_acc

In [ ]:
model.compute_patterns()
model.plot_patterns(sensor_layout='Vectorview-mag', sorting='best', spectra=False, scale=True)

In [ ]:
model.plot_waveforms(tmin= -.2)

In [ ]:
model.plot_spectra(fs=500, sorting='best', norm_spectra='welch')

# 2. VARCNN

In [ ]:
model = mneflow.models.VARCNN(dataset, optimizer, lf_params)
model.build()

model.train(n_iter=30000, eval_step=500, min_delta=1e-6, early_stopping=6)
model.plot_hist()

# 3. LFLSTM

In [ ]:
model = mneflow.models.LFLSTM(dataset, optimizer, lf_params)
model.build()

model.train(n_iter=30000, eval_step=500, min_delta=1e-6, early_stopping=6)

In [ ]:
model.compute_patterns()
model.plot_patterns(sensor_layout='Vectorview-mag', sorting='best', spectra=False, scale=True)

In [ ]:
model.plot_waveforms(tmin= -.2)

In [ ]:
model.plot_spectra(fs=500, sorting='best', norm_spectra='welch')